Importamos las librerias necesarias

In [1]:
import findspark
findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType
from pyspark.sql import Window
import pyspark.sql.functions as F
import numpy as np
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
import time


#path del archivo postgresql-42.1.4.jar
dir = "/Users/jasonsolano/Documents/BigData/BigDataTEC/Clase1/DB/"
spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", dir+"postgresql-42.1.4.jar") \
    .config("spark.executor.extraClassPath", dir+"postgresql-42.1.4.jar") \
    .getOrCreate()

start = time.time()

Importamos los datos limpios de postgres

In [2]:
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/ImmoDataBase") \
    .option("user", "postgres") \
    .option("password", "102800") \
    .option("dbtable", "apartmentrentalclean") \
    .load()

df.limit(5).toPandas().head()

,regio1,serviceCharge,heatingType,telekomTvOffer,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,...,typeOfFlat,geo_plz,noRooms,floor,noRoomsRange,garden,livingSpaceRange,regio2,regio3,date
0,Nordrhein_Westfalen,100.00,central_heating,ONE_YEAR_FREE,False,False,29,3.52,2.4,429.00,...,ground_floor,45329,2.0,0.0,2,False,2,Essen,Karnap,May19
1,Nordrhein_Westfalen,84.00,central_heating,ONE_YEAR_FREE,False,True,4,4.92,2.4,578.00,...,roof_storey,48282,3.0,2.0,3,False,3,Steinfurt_Kreis,Emsdetten,Sep18
2,Nordrhein_Westfalen,115.00,night_storage_heater,ONE_YEAR_FREE,False,False,11,4.69,40.0,365.00,...,roof_storey,46242,3.0,3.0,3,False,2,Bottrop,Lehmkuhle,Sep18
3,Sachsen_Anhalt,33.00,self_contained_central_heating,ONE_YEAR_FREE,False,False,9,0.93,40.0,358.00,...,roof_storey,39218,2.0,0.0,2,False,2,Salzlandkreis,Schönebeck_Elbe,May19
4,Sachsen,169.75,gas_heating,ONE_YEAR_FREE,False,True,9,-1.89,2.4,449.49,...,apartment,9126,2.0,2.0,2,False,2,Chemnitz,Bernsdorf,May19


Se imprime el esquema

In [3]:
df.printSchema()

root
 |-- regio1: string (nullable = true)
 |-- serviceCharge: double (nullable = true)
 |-- heatingType: string (nullable = true)
 |-- telekomTvOffer: string (nullable = true)
 |-- newlyConst: boolean (nullable = true)
 |-- balcony: boolean (nullable = true)
 |-- picturecount: integer (nullable = true)
 |-- pricetrend: double (nullable = true)
 |-- telekomUploadSpeed: double (nullable = true)
 |-- totalRent: double (nullable = true)
 |-- firingTypes: string (nullable = true)
 |-- hasKitchen: boolean (nullable = true)
 |-- geo_bln: string (nullable = true)
 |-- cellar: boolean (nullable = true)
 |-- yearConstructedRange: double (nullable = true)
 |-- baseRent: double (nullable = true)
 |-- livingSpace: double (nullable = true)
 |-- geo_krs: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- lift: boolean (nullable = true)
 |-- baseRentRange: integer (nullable = true)
 |-- typeOfFlat: string (nullable = true)
 |-- geo_plz: integer (nullable = true)
 |-- noRooms: doub

Creamos la siguiente funcioon para normalizar las columnas numericas

In [4]:
def normalizeColumn(df,listColumnName):
    unlist = udf(lambda x: round(float(list(x)[0]),10), DoubleType())

    for i in listColumnName:
       
        assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

        scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

        pipeline = Pipeline(stages=[assembler, scaler])

        # Fitting pipeline on dataframe
        df = pipeline.fit(df).transform(df).withColumn(i, unlist(i+"_Scaled")).drop(i+"_Vect")
        df = df.drop(i+"_Scaled")
    return df


Probamos la funcion para la normalizacion con solo una columna

Primero vemos la columna serviceCharge sin normalizacion

In [5]:
datasetSize = df.count()

df.groupBy('serviceCharge').count().sort(col("count").desc()).withColumn('perc_of_count_total', (F.col('count') / datasetSize)) \
  .toPandas().head(20)

,serviceCharge,count,perc_of_count_total
0,150.00,9009,0.053431
1,100.00,8066,0.047838
2,200.00,6811,0.040395
3,120.00,6615,0.039233
4,130.00,4809,0.028522
5,140.00,4587,0.027205
6,180.00,4202,0.024922
7,80.00,4148,0.024601
8,110.00,3927,0.023291
9,250.00,3914,0.023213


## Aplicamos la normalizacion con la funcion presentada anteriormente

In [6]:
numericColumns = list()
numericColumns.append('serviceCharge')
df = normalizeColumn(df,numericColumns)

Volvemos a consultar la columna para ver la normalizacion

In [7]:
df.groupBy('serviceCharge').count().sort(col("count").desc()).withColumn('perc_of_count_total', (F.col('count') / datasetSize)) \
  .toPandas().head(20)

,serviceCharge,count,perc_of_count_total
0,0.001027,9009,0.053431
1,0.000684,8066,0.047838
2,0.001369,6811,0.040395
3,0.000821,6615,0.039233
4,0.000890,4809,0.028522
5,0.000958,4587,0.027205
6,0.001232,4202,0.024922
7,0.000548,4148,0.024601
8,0.000753,3927,0.023291
9,0.001711,3914,0.023213


Una vez que se valida la normalizacion para dichos datos, correspondemos a realizarlo en las demas columnas numericas

# Normalizacion de columnas numericas

In [8]:
# Columnas

columnas = ['picturecount','pricetrend','telekomUploadSpeed','totalRent','yearConstructedRange'\
           ,'baseRent','livingSpace','baseRentRange','geo_plz','noRooms','floor','noRoomsRange',
           'livingSpaceRange']

print("Numero de columnas: ",len(columnas))

Numero de columnas:  13


## Aplicamos normalización de columnas

In [9]:
df = normalizeColumn(df,columnas)

In [10]:
df.select(columnas).toPandas().head()

,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructedRange,baseRent,livingSpace,baseRentRange,geo_plz,noRooms,floor,noRoomsRange,livingSpaceRange
0,0.266055,0.526775,0.014141,0.000027,0.000,0.000266,0.000470,0.125,0.448601,0.001789,0.001,0.25,0.166667
1,0.036697,0.584890,0.014141,0.000037,0.500,0.000372,0.000675,0.250,0.478385,0.003578,0.003,0.50,0.333333
2,0.100917,0.575342,0.393939,0.000023,0.125,0.000203,0.000396,0.000,0.457810,0.003578,0.004,0.50,0.166667
3,0.082569,0.419261,0.393939,0.000023,0.125,0.000168,0.000414,0.000,0.386965,0.001789,0.001,0.25,0.166667
4,0.082569,0.302200,0.014141,0.000029,0.000,0.000227,0.000545,0.000,0.083453,0.001789,0.003,0.25,0.166667


Como se observa, los datos numericos se encuentran normalizados

# Iniciamos con la transformacion de columnas booleansas

In [11]:
booleanColumns = ['newlyConst','balcony','hasKitchen','cellar','lift','garden']

print("Cantidad de columnas Booleanas: ",len(booleanColumns))

Cantidad de columnas Booleanas:  6


Función para remplazar columnas booleanas

In [12]:
def replaceBooleans(df,listColumns):
    for i in listColumns:
        df = df.withColumn(i,F.when((col(i) == True),1).otherwise(0))
    return df

In [13]:
df = replaceBooleans(df,booleanColumns)

Observamos la transformación`

In [14]:
df.select(booleanColumns).toPandas().head()

,newlyConst,balcony,hasKitchen,cellar,lift,garden
0,0,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,1,0,0
4,0,1,1,1,0,0


# Remplazamos variables categoricas por OneHotEncoding

Creamos la función para remplazar valores categoricos

In [15]:
def createCategoricalVects(df,columns):
    for i in columns:
        temp_indexer = StringIndexer(inputCol=i, outputCol=i+"_index").fit(df)
        df = temp_indexer.transform(df)
        temp_encoder = OneHotEncoder(inputCol=i+"_index", outputCol=i+"_vec")
        df = temp_encoder.transform(df)
        df = df.drop(i+"_index")
        print("Columna Procesada:",i,"Tammano",df.groupBy(i).count().count())
        df = df.drop(i)
    return df

Dicha función detecta si una columna tiene bastantes categorias para borrarla

Al observar la gran cantidad de columnas que se generan con diferentes datos categoricos, se evalua solo aceptar columnas categoricas con no más de 50 datos categoricos diferentes

In [16]:
def deleteBigColumns(df,columns,limit):
    goodColumns = list()
    for i in columns:
        size = df.groupBy(i).count().count()
        print(i,size)
        if(size > limit):
            print("Columna borrada:",i,"Tamanno:", size) 
            df = df.drop(i)
        else:
            goodColumns.append(i)
    return df,goodColumns

## Procesamos las columnas categoricas

In [17]:
categoryList = ['regio1','heatingType','geo_bln','geo_krs','condition','typeOfFlat','regio2','regio3','date','firingTypes','telekomTvOffer']
df,categoryList = deleteBigColumns(df,categoryList,110)
print('Procesamiento de las columnas: ')
df = createCategoricalVects(df,categoryList)


regio1 16
heatingType 13
geo_bln 16
geo_krs 419
Columna borrada: geo_krs Tamanno: 419
condition 10
typeOfFlat 10
regio2 419
Columna borrada: regio2 Tamanno: 419
regio3 7794
Columna borrada: regio3 Tamanno: 7794
date 3
firingTypes 102
telekomTvOffer 3
Procesamiento de las columnas: 
Columna Procesada: regio1 Tammano 16
Columna Procesada: heatingType Tammano 13
Columna Procesada: geo_bln Tammano 16
Columna Procesada: condition Tammano 10
Columna Procesada: typeOfFlat Tammano 10
Columna Procesada: date Tammano 3
Columna Procesada: firingTypes Tammano 102
Columna Procesada: telekomTvOffer Tammano 3


## Observamos los nuevos datos categoricos

In [18]:
vectCat = [i+"_vec" for i in categoryList]
df.select(vectCat).show()

+---------------+---------------+---------------+-------------+--------------+-------------+----------------+------------------+
|     regio1_vec|heatingType_vec|    geo_bln_vec|condition_vec|typeOfFlat_vec|     date_vec| firingTypes_vec|telekomTvOffer_vec|
+---------------+---------------+---------------+-------------+--------------+-------------+----------------+------------------+
| (15,[1],[1.0])| (12,[0],[1.0])| (15,[1],[1.0])|(9,[6],[1.0])| (9,[2],[1.0])|(2,[0],[1.0])| (101,[5],[1.0])|     (2,[0],[1.0])|
| (15,[1],[1.0])| (12,[0],[1.0])| (15,[1],[1.0])|(9,[0],[1.0])| (9,[1],[1.0])|    (2,[],[])| (101,[0],[1.0])|     (2,[0],[1.0])|
| (15,[1],[1.0])| (12,[8],[1.0])| (15,[1],[1.0])|(9,[0],[1.0])| (9,[1],[1.0])|    (2,[],[])| (101,[5],[1.0])|     (2,[0],[1.0])|
| (15,[2],[1.0])| (12,[4],[1.0])| (15,[2],[1.0])|(9,[0],[1.0])| (9,[1],[1.0])|(2,[0],[1.0])| (101,[0],[1.0])|     (2,[0],[1.0])|
| (15,[0],[1.0])| (12,[2],[1.0])| (15,[0],[1.0])|(9,[2],[1.0])| (9,[0],[1.0])|(2,[0],[1.0])| (101

# Mostramos la columnas que utilizamos para el training

In [19]:
datasetColumns = df.columns
datasetColumns.remove('totalRent')
print(datasetColumns)



['serviceCharge', 'newlyConst', 'balcony', 'picturecount', 'pricetrend', 'telekomUploadSpeed', 'hasKitchen', 'cellar', 'yearConstructedRange', 'baseRent', 'livingSpace', 'lift', 'baseRentRange', 'geo_plz', 'noRooms', 'floor', 'noRoomsRange', 'garden', 'livingSpaceRange', 'regio1_vec', 'heatingType_vec', 'geo_bln_vec', 'condition_vec', 'typeOfFlat_vec', 'date_vec', 'firingTypes_vec', 'telekomTvOffer_vec']


In [20]:
df.select(datasetColumns).show()

+-------------+----------+-------+------------+------------+------------------+----------+------+--------------------+-----------+-----------+----+-------------+------------+------------+-----+------------+------+----------------+---------------+---------------+---------------+-------------+--------------+-------------+----------------+------------------+
|serviceCharge|newlyConst|balcony|picturecount|  pricetrend|telekomUploadSpeed|hasKitchen|cellar|yearConstructedRange|   baseRent|livingSpace|lift|baseRentRange|     geo_plz|     noRooms|floor|noRoomsRange|garden|livingSpaceRange|     regio1_vec|heatingType_vec|    geo_bln_vec|condition_vec|typeOfFlat_vec|     date_vec| firingTypes_vec|telekomTvOffer_vec|
+-------------+----------+-------+------------+------------+------------------+----------+------+--------------------+-----------+-----------+----+-------------+------------+------------+-----+------------+------+----------------+---------------+---------------+---------------+------

# Creamos la columna features

In [21]:
df_assembler = VectorAssembler(inputCols = datasetColumns,outputCol="features")
df = df_assembler.transform(df)
df = df.select(['features','totalRent'])


In [22]:
df = df.withColumnRenamed('totalRent','label')

In [23]:

df.select(['features','label']).show()


+--------------------+----------+
|            features|     label|
+--------------------+----------+
|(184,[0,3,4,5,9,1...|2.72354E-5|
|(184,[0,2,3,4,5,8...|3.66948E-5|
|(184,[0,3,4,5,8,9...|2.31723E-5|
|(184,[0,3,4,5,7,8...|2.27279E-5|
|(184,[0,2,3,4,5,6...|2.85363E-5|
|(184,[0,3,4,5,7,8...|3.12985E-5|
|(184,[0,2,3,4,5,7...|2.76049E-5|
|(184,[0,2,3,4,5,9...|2.11091E-5|
|(184,[0,3,4,5,7,8...|3.19937E-5|
|(184,[0,2,3,4,5,6...| 6.5073E-5|
|(184,[0,3,4,5,8,9...|3.00923E-5|
|(184,[0,2,3,4,5,7...| 6.7295E-5|
|(184,[0,2,3,4,5,7...|3.36475E-5|
|(184,[0,2,3,4,5,8...|3.74567E-5|
|(184,[0,3,4,5,8,9...|2.69815E-5|
|(184,[0,2,3,4,5,6...|7.20565E-5|
|(184,[0,3,4,5,7,8...|2.53944E-5|
|(184,[0,2,3,4,5,6...|3.07906E-5|
|(184,[0,2,3,4,5,6...|8.17197E-5|
|(184,[0,2,3,4,5,7...|2.66641E-5|
+--------------------+----------+
only showing top 20 rows



In [24]:
df.select("features").show(20,False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                     |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(184,[0,3,4,5,9,10,12,13,14,15,16,18,20,34,47,67,72,79,86,182],[6.843784E-4,0.2660550459,0.5267745953,0.0141414141,2.664902E-4,4.698905E-4,0.125,0.448601053,0.0017889088,0.001,0.25,0.1666666667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])    

# Dividimos el dataset para entrenamiento y prueba

In [25]:
train,test = df.randomSplit([0.7,0.3])

# Inicamos con el entrenamiento

In [26]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

## Iniciamos con el algoritmo Random Forest Regressor

In [27]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [28]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

evaluator = RegressionEvaluator()

rf = RandomForestRegressor()
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2,3,5,7])
             .addGrid(rf.maxBins, [5,10,20,25 ])
             .addGrid(rf.numTrees, [5,6,7])
             .build())
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cv_model = cv.fit(train)

In [29]:
best_rf_model = cv_model.bestModel
model_predictions = best_rf_model.transform(test)

In [30]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {round(float(dt_r2),3)}')

# RMSE value of the model on test data 
dt_evaluatorRMSE = RegressionEvaluator(metricName='rmse')
dt_RMSE = dt_evaluatorRMSE.evaluate(model_predictions)
print(f'The RMSE value of DecisionTreeRegressor is {round(float(dt_RMSE),3)}')

The r-square value of DecisionTreeRegressor is -0.202
The RMSE value of DecisionTreeRegressor is 0.0


### imprimimos el valor R2:

### imprimimos el valor de meanSquaredError

## Probamos el Algoritmo Decision Tree Regressor

## Utilizamos el algoritmo Random Forest Regressor

In [31]:
end = time.time()
print("Tiempo Transcurrido en minutos: ",(end-start)/60)

Tiempo Transcurrido en minutos:  22.118071222305296


# Conclusión

Las pruebas se realizaron con un dataset 30% para validación y 70% para entrenamiento con un total de 168609 registros


Como se puede observar con el algoritmo Isotonic Regression presenta un resultado muy deficiente, 

En cuanto a los modelos  Gradient-Boosted Tree Regressor y Decision Tree Regressor el R_2 presenta un muy buen resultado, sin ser mayormente intervenido con hyperparametros por lo cual se descartan pensando que presentan overfitting

Para el caso de los modelos con los algoritmos Random Forest Regressor y Linear Regression presentan el resultado R_2 con un resultado elevado pero no tanto como los mencionadas anteriormente.

Por lo tanto se realizara en otra Jupyternote la optimización del algortimo Random Forest Regressor, que es un algoritmo mas complejo que puede permitir una mejor modelo de predicción